# Algılayıcı ile Çok Sınıflı Sınıflandırma

[Yeni Başlayanlar için Yapay Zeka Müfredatı](https://github.com/microsoft/ai-for-beginners)'ndan Laboratuvar Ödevi.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

Dersten aşağıdaki algılayıcı eğitim kodunu kullanabilirsiniz:

In [ ]:
def train(positive_examples, negative_examples, num_iterations = 100):
    num_dims = positive_examples.shape[1]
    weights = np.zeros((num_dims,1)) # ağırlıkları ilkle
    
    pos_count = positive_examples.shape[0]
    neg_count = negative_examples.shape[0]
    
    report_frequency = 10
    
    for i in range(num_iterations):
        pos = random.choice(positive_examples)
        neg = random.choice(negative_examples)

        z = np.dot(pos, weights)   
        if z < 0:
            weights = weights + pos.reshape(weights.shape)

        z  = np.dot(neg, weights)
        if z >= 0:
            weights = weights - neg.reshape(weights.shape)
            
        if i % report_frequency == 0:             
            pos_out = np.dot(positive_examples, weights)
            neg_out = np.dot(negative_examples, weights)        
            pos_correct = (pos_out >= 0).sum() / float(pos_count)
            neg_correct = (neg_out < 0).sum() / float(neg_count)
            print("Yineleme={}, pozitif doğruluk={}, negatif doğruluk={}".format(i,pos_correct,neg_correct))

    return weights

In [ ]:
def accuracy(weights, test_x, test_labels):
    res = np.dot(np.c_[test_x,np.ones(len(test_x))],weights)
    return (res.reshape(test_labels.shape)*test_labels>=0).sum()/float(len(test_labels))

accuracy(wts, test_x, test_labels)

### Veri Kümesini Okuma

Bu kod, veri kümesini internetteki depodan indirir. AI Müfredat deposunun `/data` dizininden veri kümesini manuel olarak da kopyalayabilirsiniz.

In [ ]:
!rm *.pkl
!wget https://raw.githubusercontent.com/microsoft/AI-For-Beginners/main/data/mnist.pkl.gz
!gzip -d mnist.pkl.gz

In [ ]:
with open('mnist.pkl', 'rb') as mnist_pickle:
    MNIST = pickle.load(mnist_pickle)

In [ ]:
print(MNIST['Train']['Features'][0][130:180])
print(MNIST['Train']['Labels'][0])
features = MNIST['Train']['Features'].astype(np.float32) / 256.0
labels = MNIST['Train']['Labels']
fig = plt.figure(figsize=(10,5))
for i in range(10):
    ax = fig.add_subplot(1,10,i+1)
    plt.imshow(features[i].reshape(28,28))
plt.show()

İki rakamlı sınıflandırma için *bire karşı diğeri* veri kümesi oluşturma kodu. *Bire karşı hepsi* veri kümesi oluşturmak için bu kodu değiştirmeniz gerekir.

In [ ]:
def set_mnist_pos_neg(positive_label, negative_label):
    positive_indices = [i for i, j in enumerate(MNIST['Train']['Labels']) 
                          if j == positive_label]
    negative_indices = [i for i, j in enumerate(MNIST['Train']['Labels']) 
                          if j == negative_label]

    positive_images = MNIST['Train']['Features'][positive_indices]
    negative_images = MNIST['Train']['Features'][negative_indices]

    return positive_images, negative_images

Şimdi yapmanız gerekenler:
1. Tüm rakamlar için 10 *bire karşı hepsi* veri kümesi oluşturun.
1. 10 algılayıcı eğitin.
1. Rakam sınıflandırmasını gerçekleştirmek için `classify` (sınıflandırma) işlevini tanımlayın.
1. Sınıflandırmanın doğruluğunu ölçün ve *hata matrisi*ni yazdırın.
1. [İsteğe bağlı] Bir matris çarpımı kullanarak sınıflandırmayı gerçekleştiren gelişmiş `classify` işlevini oluşturun.